In [1]:
import pandas as pd
import recordlinkage

# 1) Caricamento e pre-elaborazione dati
schema_file = "main_outputs/final_mediated_schema.csv"
df = pd.read_csv(schema_file)

# Seleziona alcune colonne di esempio (regola pure se vuoi tenere fuori la city)
df = df[['company_name', 'industry', 'headquarters_country']]
df.dropna(subset=['company_name'], inplace=True)

print(f"[Script con similarità su 'industry'] Numero di record iniziali: {len(df)}")

# 2) Creazione di un blocking basato su industry + sortedNeighbourhood sul name
indexer = recordlinkage.Index()
indexer.add(recordlinkage.index.Block('industry'))
indexer.add(recordlinkage.index.SortedNeighbourhood('company_name', window=3))

candidate_pairs = indexer.index(df)
print(f"[Script con similarità su 'industry'] Numero di coppie candidate: {len(candidate_pairs)}")

# 3) Definizione delle regole di confronto
compare = recordlinkage.Compare()

# Similarità sul company_name (jarowinkler)
compare.string('company_name', 
               'company_name', 
               method='jarowinkler', 
               label='name_sim')

# Similarità su industry (jarowinkler)
compare.string('industry', 
               'industry', 
               method='jarowinkler', 
               label='industry_sim')

# Similarità su headquarters_country (jarowinkler)
compare.string('headquarters_country',
                'headquarters_country',
                method='jarowinkler',
                label='country_sim')

# 4) Calcolo della matrice di similarità
similarity_matrix = compare.compute(candidate_pairs, df)
print(f"[Script con similarità su 'industry'] Dimensioni della similarity_matrix: {similarity_matrix.shape}")

# 5) Definizione regole di matching
#    Esempio di condizione: 
#     1) name_sim tra 0.80 e 0.92
#     2) industry_sim > 0.85  (scegli tu la soglia opportuna)
matches = similarity_matrix[
    (similarity_matrix['name_sim'] > 0.80) & (similarity_matrix['name_sim'] < 0.92) |
    (similarity_matrix['name_sim'] > 0.9) & (similarity_matrix['industry_sim'] > 0.5) &
     (similarity_matrix['country_sim'] >0.5)
]

print(f"[Script con similarità su 'industry'] Numero di coppie finali considerate 'match': {len(matches)}")

# 6) Recupero dei record corrispondenti (join con df originale)
matches = matches.reset_index()
matches.rename(columns={'level_0': 'id_left', 'level_1': 'id_right'}, inplace=True)
matches = matches.merge(df, left_on='id_left', right_index=True, how='left', suffixes=('', '_left'))
matches = matches.merge(df, left_on='id_right', right_index=True, how='left', suffixes=('_left', '_right'))

# Eliminiamo le righe in cui i campi company_name sono identici (stesso record)
matches = matches[matches['company_name_left'] != matches['company_name_right']]

# 7) Salvataggio in CSV
output_file = "matched_companies_blocking_1.csv"
matches.to_csv(output_file, index=False)
print(f"[Script con similarità su 'industry'] ✅ File '{output_file}' generato!")


C:\Users\luca-\AppData\Local\Temp\ipykernel_2556\3508856985.py:6: DtypeWarning: Columns (2,5,6,7,9,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(schema_file)


[Script con similarità su 'industry'] Numero di record iniziali: 75793
[Script con similarità su 'industry'] Numero di coppie candidate: 7271053
[Script con similarità su 'industry'] Dimensioni della similarity_matrix: (7271053, 3)
[Script con similarità su 'industry'] Numero di coppie finali considerate 'match': 83709
[Script con similarità su 'industry'] ✅ File 'matched_companies_blocking_1.csv' generato!


In [ ]:
import pandas as pd
import json
import numpy as np

# Path dei file (aggiorna con i percorsi reali)
pairwise_file = 'matched_companies_blocking_1.csv'
groundtruth_file = 'data/ground_truth.csv'
output_json_file = 'filtered_ground_truth_1.json'

# Caricamento dei file
pairwise = pd.read_csv(pairwise_file)
groundtruth = pd.read_csv(groundtruth_file)

# Funzione di supporto per normalizzare i nomi di società (rimuove spazi e usa lowercase)
def normalize_name(name):
    return name.strip().lower() if isinstance(name, str) else str(name).lower()

# Funzione per ottenere una tupla ordinata, così che l'ordine non conti
def normalize_pair(row, col1, col2):
    left = normalize_name(row[col1])
    right = normalize_name(row[col2])
    return tuple(sorted([left, right]))

# Creiamo la colonna 'normalized_pair' in entrambi i dataset
pairwise['normalized_pair'] = pairwise.apply(
    lambda x: normalize_pair(x, 'company_name_left', 'company_name_right'), axis=1
)
groundtruth['normalized_pair'] = groundtruth.apply(
    lambda x: normalize_pair(x, 'company_a', 'company_b'), axis=1
)

# Convertiamo in insiemi per confronto
pairwise_set = set(pairwise['normalized_pair'])
groundtruth_set = set(groundtruth['normalized_pair'])

# Intersezione tra le coppie del dataset pairwise e quelle della ground truth (tutte, indipendentemente dalla label)
intersection = pairwise_set.intersection(groundtruth_set)

# Filtriamo la ground truth per ottenere solo le coppie con label 1 (cioè i match veri)
groundtruth_true_set = set(groundtruth[groundtruth['label'] == 1]['normalized_pair'])

# Calcoliamo il numero di coppie dell'intersezione che hanno label 1
tp = len(intersection.intersection(groundtruth_true_set))

# Calcolo delle metriche:
precision = tp / len(intersection) if len(intersection) else 0
recall = tp / len(groundtruth_true_set) if len(groundtruth_true_set) else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

# Output dei risultati
print(f"Numero di coppie in pairwise: {len(pairwise_set)}")
print(f"Numero di coppie in ground truth: {len(groundtruth_set)}")
print(f'Numero di coppie in ground truth con label a 1: {len(groundtruth_true_set)}')
print(f"Numero di coppie in intersezione: {len(intersection)}")
print(f"Numero di coppie nell'intersezione con label 1: {tp}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

# Creazione del JSON filtrato
filtered_gt = {"ground_truth": []}

# Iteriamo sulla ground truth e selezioniamo solo le coppie nell'intersezione
for _, row in groundtruth.iterrows():
    normalized = normalize_pair(row, 'company_a', 'company_b')
    if normalized in intersection:
        gt_entry = {
            "pairs": [
                {
                    "company-name": row["company_a"],
                    "industry": row["industry_a"].split("; ") if isinstance(row["industry_a"], str) else None,
                    "country": None if pd.isna(row["country_a"]) else row["country_a"],
                    "source": None if pd.isna(row["source_a"]) else row["source_a"]
                },
                {
                    "company-name": row["company_b"],
                    "industry": row["industry_b"].split("; ") if isinstance(row["industry_b"], str) else None,
                    "country": None if pd.isna(row["country_b"]) else row["country_b"],
                    "source": None if pd.isna(row["source_b"]) else row["source_b"]
                }
            ],
            "match": True if row["label"] == 1 else False
        }
        filtered_gt["ground_truth"].append(gt_entry)

# Salvataggio su file JSON
with open(output_json_file, "w", encoding="utf-8") as json_file:
    json.dump(filtered_gt, json_file, indent=4, ensure_ascii=False)

print(f"✅ File JSON '{output_json_file}' generato con successo!")


Numero di coppie in pairwise: 69766
Numero di coppie in ground truth: 200
Numero di coppie in ground truth con label a 1: 100
Numero di coppie in intersezione: 169
Numero di coppie nell'intersezione con label 1: 76
Precision: 0.4497
Recall: 0.7600
F1-Score: 0.5651
✅ File JSON 'filtered_ground_truth_1.json' generato con successo!
